# Naive Bayes (the easy way)

We'll cheat by using sklearn.naive_bayes to train a spam classifier! Most of the code is just loading our training data into a pandas DataFrame that we can play with:

In [25]:
import os
import io
import numpy
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

def readFiles(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            path = os.path.join(root, filename)

            inBody = False
            lines = []
            f = io.open(path, 'r', encoding='latin1')
            for line in f:
                if inBody:
                    lines.append(line)
                elif line == '\n':
                    inBody = True
            f.close()
            message = '\n'.join(lines)
            yield path, message


def dataFrameFromDirectory(path, classification):
    rows = []
    index = []
    for filename, message in readFiles(path):
        rows.append({'message': message, 'class': classification})
        index.append(filename)

    return DataFrame(rows, index=index)

data = DataFrame({'message': [], 'class': []})

data = data.append(dataFrameFromDirectory('/Users/nielsipsen/proj/python/python-udemy/prog/DataScience-Python3/emails/spam', 'spam'))
data = data.append(dataFrameFromDirectory('/Users/nielsipsen/proj/python/python-udemy/prog/DataScience-Python3/emails/emails/ham', 'ham'))

for root, dirnames, filenames in os.walk('/Users/nielsipsen/proj/python/python-udemy/prog/DataScience-Python3/emails/spam'):
    print(root)


/Users/nielsipsen/proj/python/python-udemy/prog/DataScience-Python3/emails/spam


Let's have a look at that DataFrame:

In [26]:
data.head()

,class,message
/Users/nielsipsen/proj/python/python-udemy/prog/DataScience-Python3/emails/spam/00001.7848dde101aa985090474a91ec93fcf0,spam,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 Tr..."
/Users/nielsipsen/proj/python/python-udemy/prog/DataScience-Python3/emails/spam/00002.d94f1b97e48ed3b553b3508d116e6a09,spam,1) Fight The Risk of Cancer!\n\nhttp://www.adc...
/Users/nielsipsen/proj/python/python-udemy/prog/DataScience-Python3/emails/spam/00003.2ee33bc6eacdb11f38d052c44819ba6c,spam,1) Fight The Risk of Cancer!\n\nhttp://www.adc...
/Users/nielsipsen/proj/python/python-udemy/prog/DataScience-Python3/emails/spam/00004.eac8de8d759b7e74154f142194282724,spam,##############################################...
/Users/nielsipsen/proj/python/python-udemy/prog/DataScience-Python3/emails/spam/00005.57696a39d7d84318ce497886896bf90d,spam,I thought you might like these:\n\n1) Slim Dow...


Now we will use a CountVectorizer to split up each message into its list of words, and throw that into a MultinomialNB classifier. Call fit() and we've got a trained spam filter ready to go! It's just that easy.

In [27]:
vectorizer = CountVectorizer()
counts = vectorizer.fit_transform(data['message'].values)
print(counts)


  (0, 12496)	1
  (0, 17635)	5
  (0, 27488)	1
  (0, 35042)	1
  (0, 12977)	1
  (0, 32767)	1
  (0, 13775)	1
  (0, 17025)	2
  (0, 22954)	2
  (0, 10959)	2
  (0, 2233)	28
  (0, 32210)	1
  (0, 10181)	1
  (0, 2421)	1
  (0, 660)	1
  (0, 17643)	2
  (0, 13991)	1
  (0, 2336)	1
  (0, 37605)	1
  (0, 23530)	1
  (0, 0)	1
  (0, 1424)	1
  (0, 512)	1
  (0, 23930)	1
  (0, 2351)	1
  :	:
  (499, 39360)	1
  (499, 38874)	1
  (499, 39347)	1
  (499, 39115)	1
  (499, 23802)	1
  (499, 16712)	2
  (499, 39332)	2
  (499, 39331)	2
  (499, 39507)	2
  (499, 38872)	2
  (499, 39386)	1
  (499, 39452)	1
  (499, 39283)	1
  (499, 38900)	1
  (499, 38955)	1
  (499, 38883)	1
  (499, 39328)	1
  (499, 38958)	1
  (499, 39468)	1
  (499, 39090)	1
  (499, 38957)	1
  (499, 38823)	1
  (499, 39503)	1
  (499, 22391)	1
  (499, 29868)	1


In [29]:
classifier = MultinomialNB()
targets = data['class'].values
classifier.fit(counts, targets)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

Let's try it out:

In [30]:
examples = ['Free Viagra now!!!', "Hi Bob, how about a game of golf tomorrow?"]
example_counts = vectorizer.transform(examples)
predictions = classifier.predict(example_counts)
predictions

array(['spam', 'spam'], 
      dtype='<U4')

## Activity

Our data set is small, so our spam classifier isn't actually very good. Try running some different test emails through it and see if you get the results you expect.

If you really want to challenge yourself, try applying train/test to this spam classifier - see how well it can predict some subset of the ham and spam emails.

In [31]:
examples = ['Free','are you free tonight? we could go out golfing or something','Hi Bob, how about a game of golf tomorrow? ']
example_counts = vectorizer.transform(examples)
predictions = classifier.predict(example_counts)
predictions

array(['spam', 'spam', 'spam'], 
      dtype='<U4')